In [1]:
from datetime import date
from pathlib import Path

from loguru import logger
import sentier_data_tools as sdt

import os 

import brightway2 as bw
import lca_algebraic as agb
import numpy as np
from sympy import init_printing
import bw2io
import bw2data as bd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', 500)
from sentier_data_tools import (
    Demand,
    Flow,
    FlowIRI,
    GeonamesIRI,
    ModelTermIRI,
    DatasetKind,
    ProductIRI,
    SentierModel,
)
# Pretty print for Sympy
init_printing()

In [140]:
bw.projects.set_current('PCB_Project')

bw.databases

Initial setup already done, skipping


Databases dictionary with 3 object(s):
	MyForeground
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [141]:
EcoInvent = "ecoinvent-3.10-cutoff"
USER_DB = 'MyForeground'
#agb.setForeground(DB_Buck)
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)
# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

[WARNING] Db MyForeground was here. Reseting it


,backend,nb_activities,type
name,,,
ecoinvent-3.10-biosphere,sqlite,4363,biosphere
ecoinvent-3.10-cutoff,sqlite,23523,background
MyForeground,sqlite,0,foreground


In [10]:
class PCB_Model(SentierModel):
    provides = {
        ProductIRI(
            "http://data.europa.eu/xsp/cn2024/853400000080"
        ): "Printed Circuit Board",
    }
    aliases = {
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/layers"
        ): "number_of_layer",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/prepregRef"
        ): "prepreg_ref",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/board_thickness"
        ): "board_thickness",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/mounting_type"
        ): "mounting_type",
        
        ProductIRI("https://vocab.sentier.dev/products/pcb"): "Electronic_factory",
    }

    def get_electrolysis_inventory(self) -> None:
        bom_electrolysis = self.get_model_data(
            self, product=self.hydrogen, kind=DatasetKind.BOM
        )

    def run(self) -> tuple[list[Demand], list[Flow]]:
        self.prepare()

In [65]:
def create_pcb_local_datastorage(reset: bool = True):
    if reset:
        sdt.reset_local_database()

    df = pd.read_excel('Parameters_df.xlsx')
    assert len(COLUMNS) == len(UNITS)
    assert len(COLUMNS) == len(df.columns) -4

    metadata = sdt.Datapackage(
        name="pcb production data from EcoInvent3.10",
        description="inventory taken in EcoInvent3.10 database",
        contributors=[
            {
                "title": "Yannis Rosset",
                "path": "I don't have any website !",
                "role": "author",
            },
        ],
        homepage="https://ecoquery.ecoinvent.org/3.10/cutoff/dataset/9764/documentation",
    ).metadata()
    metadata.pop("version")

    for layers, finish, thick, prepreg in TYPES:
        filtered = df[(df["layers"] == layers) & (df["finish"] == finish) & (df["thickness"] == thick) & (df["prepregID"] == prepreg)].copy()
        logger.info("Adding {} records for type {}", len(filtered), (layers, finish, thick, prepreg))
        #filtered.drop(labels=["layers"], axis="columns", inplace=True)
        #filtered.columns = COLUMNS

        sdt.Dataset(
            name=f"pcb process model",
            dataframe=filtered,
            product="pcb",
            columns=[{"iri": x, "unit": y} for x, y in zip(COLUMNS, UNITS)],
            metadata=metadata,
            location="https://www.geonames.org/6295630/",
            version=1,
            valid_from=date(2018, 1, 1),
            valid_to=date(2028, 1, 1),
        ).save()


COLUMNS = [
    "http://data.europa.eu/xsp/cn2024/271600000080", #electricity
    "https://vocab.sentier.dev/product/energy/heat",  #heat ok 
    "https://vocab.sentier.dev/flows/envo/ENVO_00003065",  #ultrapure water ok 
    "https://vocab.sentier.dev/product/electronic_factory", 
    "http://data.europa.eu/xsp/cn2024/740300000080", #copper
    "http://data.europa.eu/xsp/cn2024/701913000080", #glass fiber
    "http://data.europa.eu/xsp/cn2024/390730000080", # Epoxy
    "http://data.europa.eu/xsp/cn2024/280600000080", #HCl
    "http://data.europa.eu/xsp/cn2024/284700000080", #H2O2
    "http://data.europa.eu/xsp/cn2024/282739200080", #Fe (III) Cl
    "http://data.europa.eu/xsp/cn2024/710800000080", #Au
    "http://data.europa.eu/xsp/cn2024/750610000080", #Ni
    "http://data.europa.eu/xsp/cn2024/710691000080", #Ag
    "http://data.europa.eu/xsp/cn2024/250100000080", #NaCl
    "http://data.europa.eu/xsp/cn2024/281512000080", #NaOH
    "http://data.europa.eu/xsp/cn2024/280700000080", #Sulfuric acid
    "http://data.europa.eu/xsp/cn2024/290919900080",  #Solvent
    "https://vocab.sentier.dev/flows/envo/ENVO_00003096",  # water ok

]

UNITS = [
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/M3",
    "https://vocab.sentier.dev/units/unit/NUM", #part of a factory
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM", #epoxy
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
   
]
types = []
df = pd.read_excel('Parameters_df.xlsx')
for _, row in df.iterrows():
    types.append((row['layers'],row['finish'], row['thickness'],row['prepregID']))
TYPES = types




In [66]:
create_pcb_local_datastorage(True)

2024-10-11 11:37:27.223 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (1, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.244 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (2, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.260 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (4, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.275 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (6, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.290 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (8, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.306 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (10, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.318 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (12, 'Ag', 1.6, 1080)
2024-10-11 11:37:27.329 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 1 records for type (1,

In [47]:
df_treatment = pd.read_excel('Parameters_df.xlsx')
df_treatment

,layers,finish,thickness,prepregID,Electricity,Energy,ultrapure water,Electronic component factory,CU input,GF input,...,H2O2,Fe(III) Cl,Au,Ni,Ag,NaCl,NaOH,Sulfuric Acid,Solvent,Water
0,1,Ag,1.6,1080,17.9757,1.56311,49.1812,1e-12,0.140862,0.868,...,0.0722671,0.129107,0,0,0.001259,0.023105,0.768077,0.286617,0.02265,4.44369
1,2,Ag,1.6,1080,35.9514,3.12621,98.3623,1e-12,0.281723,0.868,...,0.144534,0.258215,0,0,0.001259,0.04621,1.53615,0.573233,0.0453,8.88739
2,4,Ag,1.6,1080,71.9028,6.25242,196.725,1e-12,0.563446,0.868,...,0.289068,0.516429,0,0,0.001259,0.09242,3.07231,1.14647,0.0906,17.7748
3,6,Ag,1.6,1080,107.854,9.37863,295.087,1e-12,0.845169,0.868,...,0.433603,0.774644,0,0,0.001259,0.13863,4.60846,1.7197,0.1359,26.6622
4,8,Ag,1.6,1080,143.806,12.5048,393.449,1e-12,1.12689,0.868,...,0.578137,1.03286,0,0,0.001259,0.18484,6.14462,2.29293,0.1812,35.5495
5,10,Ag,1.6,1080,179.757,15.6311,491.812,1e-12,1.40862,0.868,...,0.722671,1.29107,0,0,0.001259,0.23105,7.68077,2.86617,0.2265,44.4369
6,12,Ag,1.6,1080,215.709,18.7573,590.174,1e-12,1.69034,0.868,...,0.867205,1.54929,0,0,0.001259,0.27726,9.21693,3.4394,0.2718,53.3243
7,1,Ni/Au,1.6,1080,17.9757,1.56311,49.1812,1e-12,0.140862,0.868,...,0.0722671,0.129107,0.0007728,0.017814,0,0.023105,0.768077,0.286617,0.02265,4.44369
8,2,Ni/Au,1.6,1080,35.9514,3.12621,98.3623,1e-12,0.281723,0.868,...,0.144534,0.258215,0.0007728,0.017814,0,0.04621,1.53615,0.573233,0.0453,8.88739
9,4,Ni/Au,1.6,1080,71.9028,6.25242,196.725,1e-12,0.563446,0.868,...,0.289068,0.516429,0.0007728,0.017814,0,0.09242,3.07231,1.14647,0.0906,17.7748


In [36]:
print((6.5057+64.59338310000001)/16)

4.443692693750001


In [43]:
0.5/8960

If possible : average of 6 layer and 10 layers studies
If not : 6 layer study (EcoInvent)

- Electricity : 17.975711333333333 * nb_layers 
- Energy :  1.5631053333333333 * nb_layers
- GF : ratio_GF_Epoxy * board_thickness * 1550 (volumic mass FG)
- Epoxy : (1-ratio_GF_Epoxy) * board_thickness * 1100 (volumic mass Epoxy)
- Copper : 0.1408615829375 * nb_layers
- HCl : 0.5099911820625 * nb_layers
- H2O2 : nb_layers * 0.072267098375
- FeCl : nb_layers * 0.1291073470625
- NaCl : nb_layers * 0.023105
- NaOH : nb_layers * 0.7680773412499999
- Sulfuric Acid : nb_layers * 0.2866166794375
- Solvent : nb_layers * 0.02265
- Water : nb_layers * 4.443692693750001
- Ultrapure water :  nb_layers * 49.18115
- Pad surface : 0.4
- Gold : 0.0007728000000000001 * param_finish
- Ag : 0.0012588 * param_finish
- Ni : 0.017814 * param_finish
